- Zero Crossing Rate: Rate at which the signal changes from positive to negative or vice versa.

- Spectral centroid: Indicates at which frequency the energy of a spectrum is centered upon(Center of mass)

- Spectral Rolloff: Frequency below which a specified percentage of the total spectral energy lies.

- Spectral Contrast: Spectral peak,spectral valley and their difference in each frequency subband.

- Mel Frequency Ceptral coefficient: Set of features used to describe the overall shape of a spectral envelope.

- Chroma: How much energy of each pitch class {C,C#,D,D#,E etc} is present in the signal.

- Tonnetz: Captures harmonic relationships


In [ ]:
#Importing libraries first 
import glob
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/My Drive/data/audio')

In [ ]:
files=glob.glob('/content/drive/My Drive/data/audio/**/*.wav')
len(files)

In [ ]:
genre=[]
for f in files:
    genre.append(f.split("/")[6])

genre=pd.Series(genre)
genre.value_counts()

In [ ]:
def extract_feature(file_name):
    file=[]
    file.append(file_name)
    #Returns audio time series with sample rate
    x, sample_rate = librosa.load(file_name) 
    print ("Features :",len(x), "sampled at ", sample_rate, "hz")
    #Divide a longer time signal into shorter segments of equal length and then compute
    #the Fourier transform separately on each shorter segment (magnitude of frequency bin f at frame t)
    stft = np.abs(librosa.stft(x))
    zcr = np.mean(librosa.feature.zero_crossing_rate(x).T)
    spectral_centroid=np.mean(librosa.feature.spectral_centroid(x, sr=sample_rate)[0].T)
    spectral_rolloff= np.mean(librosa.feature.spectral_rolloff(x+0.01, sr=sample_rate)[0].T)
    spectral_contrast=np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T)
    mfccs = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=40).T)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=sample_rate).T)
    return file,zcr,spectral_centroid,spectral_rolloff,spectral_contrast,mfccs,chroma,tonnetz

In [ ]:
features=[]
for f in files:
    features.append(extract_feature(f))

In [ ]:
features[0]

In [ ]:
data=pd.DataFrame(features,columns=['file','zcr','spectral_centroid','spectral_rolloff','spectral_contrast','mfccs','chroma','tonnetz'])
data

In [ ]:
data['file']=genre

In [ ]:
data.head()

In [ ]:
import numpy as np
from sklearn import neighbors as nb
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, scale

In [ ]:
X=data.drop('file',axis=1)
y=data['file']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)

In [ ]:
enc=LabelEncoder()
target=enc.fit_transform(y_train)

In [ ]:
enc.classes_

In [ ]:
target

In [ ]:
#Scaling
train=scale(x_train,axis=0)
train=pd.DataFrame(train)
train.columns=[x_train.columns]

In [ ]:
clf=nb.KNeighborsClassifier(n_neighbors=5,n_jobs=-1,weights='distance')
clf.fit(train,target)

In [ ]:
#Scaling
test=scale(x_test,axis=0)
test=pd.DataFrame(test)
test.columns=[x_test.columns]

In [ ]:
y_test=enc.transform(y_test)

In [ ]:
clf.score(test,y_test)

In [ ]:
#GridSearch on value of k
from sklearn.model_selection import GridSearchCV
model=GridSearchCV(clf,param_grid={'n_neighbors':[3,5,7],'weights':['distance','uniform']})
model.fit(train,target)

In [ ]:
#Best estimator
model.best_estimator_

In [ ]:
model.best_params_

In [ ]:
model.best_estimator_.score(test,y_test)

In [ ]:
#Get Neighbors
knn=model.best_estimator_

In [ ]:
knn.kneighbors(test.loc[0:0,:],3)

In [ ]:
train.head()

In [ ]:
x=train.iloc[[9,0,6],:]
x

In [ ]:
#Cosine similarity
from sklearn.metrics.pairwise import euclidean_distances
euclidean_distances(test.loc[0:0,:],x)

In [ ]:
pd.Series(euclidean_distances(test.loc[0:0,:],train)[0]).sort_values()[0:3]